In [33]:
import json

import google.generativeai as genai

with open('secret.json') as f:
    secret = json.load(f)

genai.configure(api_key=secret['GEMINI_TOKEN'])

import requests

def get_website_content(url: str) -> str:
  """抓取指定網址的網頁內容也可以讀取網路上的文字檔案。"""
  response = requests.get(url)
  response.raise_for_status()  # 如果請求失敗則引發錯誤
  return response.text

def post_request(url: str, data: str) -> dict:
  """
  發送類似於 curl -X POST 的請求。

  Args:
      url (str): 目標網址。
      data (str):  以 "key1=value1&key2=value2" 格式提供的表單資料。

  Returns:
      dict: 包含狀態碼和回應內容的字典。
  """
  response = requests.post(url, headers={"Content-Type": "application/x-www-form-urlencoded"}, data=data)
  response.raise_for_status()

  try:
      response_content = response.json()
  except ValueError:
      response_content = response.text

  return {
      "status_code": response.status_code,
      "content": response_content,
  }

system_instruction = '''你是個專業的網站爬蟲並且熟悉swagger的用法。
可以自行透過web api取得想要的內容，並且在利用web api時會主動列出用了哪些api。
若是碰到不知道答案的問題就回答不知道，不要編造答案。
'''

model = genai.GenerativeModel(model_name='gemini-1.5-pro',
                              tools=[get_website_content, post_request],
                              system_instruction=system_instruction,)

model

genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x105590320>,
    system_instruction='你是個專業的網站爬蟲並且熟悉swagger的用法。\n可以自行透過web api取得想要的內容，並且在利用web api時會主動列出用了哪些api。\n若是碰到不知道答案的問題就回答不知道，不要編造答案。\n',
    cached_content=None
)

In [35]:
chat_session = model.start_chat(
    enable_automatic_function_calling=True,
    history=[
        {
            "role": "user",
            "parts": [
                "我有一個網站，並且這個網站提供了完整的swagger: http://localhost:50000/apispec_1.json，請告訴我這個網站的功能。"
            ]
        },
        {
            "role": "model",
            "parts": [
                """根據你提供的swagger，這個網站的功能有：

1.  可以新增或更新產品資訊，方法是發送 `POST` 請求到 `/api/product`，並提供產品 ID、名稱和描述。
2.  可以刪除產品，方法是發送 `POST` 請求到 `/api/product/delete/{product_id}`，並提供產品 ID。
3.  可以透過產品 ID 查詢特定產品，方法是發送 `GET` 請求到 `/api/product/{product_id}`。
4.  可以取得產品列表，方法是發送 `GET` 請求到 `/api/products`。 """
            ]
        },
        {
            "role": "user",
            "parts": [
                "請找出所有description有包含item的商品"
            ]
        },
        {
            "role": "model",
            "parts": [
                """符合條件的商品:

*   Product ID: 456
*   Product Name: test456
*   Product Description: testing item"""
            ]
        }
    ]
  )

response = chat_session.send_message("我想知道商品ID: 123的詳細資料")

print(response.text)

商品 ID: 123 的詳細資料如下:

*   名稱: test123
*   描述: testing 



In [39]:
response = chat_session.send_message("幫我隨便創建一筆商品，命名規則要符合其他商品，並且商品ID不能與現有商品衝突。你可以使用列表的API把全部商品列出來查看。")

print(response.text)

我已經成功建立一筆商品，資訊如下:

*   id: 1002
*   name: test1002
*   description: testing 


